# Catalyst Steel Segmentation



From: https://github.com/chizhu/kaggle-severstal


Applies 5 class classification including background class and then 4 class segmentation.

Classification: resnet50, efficientnet-b3 and se-resnext50.

Segmentation: Unet with resnet18, PSPNet with resnet18 and FPN with resnet50.

## Copy files from Google Drive

In [1]:
# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Steel Segmentation/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Copy python utils

In [2]:
import os

In [3]:
# copy the specified folder from google drive
def copy_support_directory( name ):  
  if os.path.exists(name): 
    !rm -r {name}    
  !mkdir {name}
  gd_dir = base_dir + name
  !cp -r '{gd_dir}/'* {name}/.  

In [4]:
copy_support_directory( 'utils' )
copy_support_directory( 'schedulers' )
copy_support_directory( 'transforms' )
copy_support_directory( 'losses' )
copy_support_directory( 'datasets' )
copy_support_directory( 'optimizers' )

## Copy python models

In [5]:
copy_support_directory( 'models' )

## Copy config files

In [6]:
copy_support_directory( 'config' )

## Copy the original image data

In [7]:
%%time

# get the image zip files using gdown as drive mapping was timing out
import gdown

# the directory containing the original competition data
data_dir = 'SteelDefect'

# test if the images are already here
if os.path.isdir(data_dir):
  !rm -r {data_dir}

!gdown --id 1fE3ITnDMGWdyckgynmLhWEN4DzJKDxtw
!mkdir -p {data_dir}
!unzip -q severstal-steel-defect-detection.zip -d {data_dir}
!rm severstal-steel-defect-detection.zip

Downloading...
From: https://drive.google.com/uc?id=1fE3ITnDMGWdyckgynmLhWEN4DzJKDxtw
To: /content/severstal-steel-defect-detection.zip
1.68GB [00:22, 75.8MB/s]
CPU times: user 226 ms, sys: 62.8 ms, total: 289 ms
Wall time: 1min 33s


In [8]:
!ls

config	  gdrive  models      sample_data  SteelDefect	utils
datasets  losses  optimizers  schedulers   transforms


## Get Required Libraries

In [9]:
# for augmentations
!pip install albumentations -q

# for pretrained segmentation models fo PyTorch
!pip install segmentation-models-pytorch -q

# for TTA
!pip install ttach==0.0.2 -q

# for Catalyst
!pip install -U catalyst -q

     |████████████████████████████████| 634kB 4.7MB/s 
     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 399kB 4.4MB/s 
     |████████████████████████████████| 317kB 12.9MB/s 
     |████████████████████████████████| 163kB 14.5MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 


In [10]:
import argparse

from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from numpy.random.mtrand import RandomState
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")

from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import CheckpointCallback, F1ScoreCallback
from utils.metrics import MultiClassAccuracyCallback

from models import CustomNet

from utils.config import load_config, save_config
from datasets import make_loader
from optimizers import get_optimizer
from losses import get_loss
from schedulers import get_scheduler
from transforms import get_transforms


In [11]:
MASKS = Path(data_dir + '/train.csv')
IMAGES = Path(data_dir + '/train_images')

gd_original_train_csv = base_dir + 'Input/orig_train.csv'

def create_original_train_csv():
  """ copy or create the original training CSV file
      - this had a single column for 'ImageId_ClassId' as opposed to sepate columns
      - additionally had an entry for each defect on each image rather than only entries
        when a defect is present
  """

  # test if the masks.csv file already exists on google drive
  if os.path.exists(gd_original_train_csv): 

    # copy the original training csv file from google drive

    print(f"Copying {gd_original_train_csv}")
    !cp '{gd_original_train_csv}' SteelDefect/.

  else:

    # need to create the masks dataframe from the training dataframe
    # - this takes about 5 minutes
    df_masks = pd.read_csv(MASKS)

    # add an 'ImageId_ClassId' column to put things back into the old format
    df_masks['ImageId_ClassId'] = df_masks.apply (lambda row: row.ImageId + '_' + str(row.ClassId) , axis=1)
    df_masks.drop(columns=['ImageId','ClassId'],inplace=True)

    # swap the column order to match the original
    columns_titles = ["ImageId_ClassId","EncodedPixels"]
    df_masks=df_masks.reindex(columns=columns_titles)

    # add blank entries for defects
    fnames = os.listdir(IMAGES)
    for fname in tqdm(fnames):
      for defect in range(1,5):
        defect_image = f"{fname}_{defect}"    
        if defect_image not in df_masks["ImageId_ClassId"].values:   
          row_df = pd.DataFrame([defect_image],columns=['ImageId_ClassId'])
          df_masks = pd.concat([row_df, df_masks], ignore_index=True)

    df_masks = df_masks.sort_values(["ImageId_ClassId"])  
    df_masks = df_masks.reset_index(drop=True) 

    # this file is 17MB in size and takes a long time to generate, so make sure its saved
    # - make sure it retains the indexes as these are the image names
    df_masks.to_csv(gd_original_train_csv, index=False)

    # save a version to the input directory for use now
    df_masks.to_csv('SteelDefect/orig_train.csv', index=False)

In [12]:
def stratified_group_k_fold(
        label: str,
        group_column: str,
        df: pd.DataFrame = None,
        file: str = None,
        n_splits=5,
        seed: int = 0
):
    random_state = RandomState(seed)

    if file is not None:
        df = pd.read_csv(file)

    labels = defaultdict(set)
    for g, l in zip(df[group_column], df[label]):
        labels[g].add(l)

    group_labels = dict()
    groups = []
    Y = []
    for k, v in labels.items():
        group_labels[k] = random_state.choice(list(v))
        Y.append(group_labels[k])
        groups.append(k)

    index = np.arange(len(group_labels))
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True,
                            random_state=random_state).split(index, Y)

    group_folds = dict()
    for i, (train, val) in enumerate(folds):
        for j in val:
            group_folds[groups[j]] = i

    res = np.zeros(len(df))
    for i, g in enumerate(df[group_column]):
        res[i] = group_folds[g]

    return res.astype(np.int)


def stratified_k_fold(
        label: str, df: pd.DataFrame = None, file: str = None, n_splits=5,
        seed: int = 0
):
    random_state = RandomState(seed)

    if file is not None:
        df = pd.read_csv(file)

    index = np.arange(df.shape[0])
    res = np.zeros(index.shape)
    folds = StratifiedKFold(n_splits=n_splits,
                            random_state=random_state,
                            shuffle=True).split(index, df[label])

    for i, (train, val) in enumerate(folds):
        res[val] = i
    return res.astype(np.int)


def split_folds(config_file):
    config = load_config(config_file)
    os.makedirs(config.work_dir, exist_ok=True)

    # test if the folds.csv file already exists on google drive
    gd_folds_csv = base_dir + 'Input/folds.csv'
    if os.path.exists(gd_folds_csv): 
    
      # copy the folds csv file from google drive
      print(f"Copying {gd_folds_csv}")
      !cp '{gd_folds_csv}' .

    else:    

      print(f"Creating folds.csv")

      df = pd.read_csv(config.data.train_df_path)
      df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
      df['ClassId'] = df['ClassId'].astype(int)
      df['exists'] = df['EncodedPixels'].notnull().astype(int)
      df['ClassId0'] = [row.ClassId if row.exists else 0 for row in df.itertuples()]
      df['fold'] = stratified_group_k_fold(
          label='ClassId0', group_column='ImageId', df=df, n_splits=config.data.params.num_folds
      )
      pv_df = df.pivot(index='ImageId', columns='ClassId', values='EncodedPixels')
      pv_df = pv_df.merge(df[['ImageId', 'fold']], on='ImageId', how='left')
      pv_df = pv_df.drop_duplicates()
      pv_df = pv_df.set_index('ImageId')
      pv_df.to_csv('folds.csv')

      # save a copy back on google drive     
      pv_df.to_csv(gd_folds_csv)


def create_folds(config_file):
    print('split train dataset for Severstal Steel Defect Detection.')    
    if config_file is None:
        raise Exception('no configuration file')
    print('load config from {}'.format(config_file))

    # make sure we have the original training csv
    create_original_train_csv()

    # split the data into folds
    split_folds(config_file)

In [13]:
create_folds('config/base_config.yml')

split train dataset for Severstal Steel Defect Detection.
load config from config/base_config.yml
Copying /content/gdrive/My Drive/Steel Segmentation/Input/orig_train.csv
Copying /content/gdrive/My Drive/Steel Segmentation/Input/folds.csv


# Classification

In [14]:
model_dir = base_dir + 'Output/Model/'

def run(config_file):
    config = load_config(config_file)

    os.makedirs(config.work_dir, exist_ok=True)
    save_config(config, config.work_dir + '/config.yml')

    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

    all_transforms = {}
    all_transforms['train'] = get_transforms(config.transforms.train)
    all_transforms['valid'] = get_transforms(config.transforms.test)

    dataloaders = {
        phase: make_loader(
            data_folder=config.data.train_dir,
            df_path=config.data.train_df_path,
            phase=phase,
            batch_size=config.train.batch_size,
            num_workers=config.num_workers,
            idx_fold=config.data.params.idx_fold,
            transforms=all_transforms[phase],
            num_classes=config.data.num_classes,
            pseudo_label_path=config.train.pseudo_label_path,
            task='cls'
        )
        for phase in ['train', 'valid']
    }

    # create model
    model = CustomNet(config.model.encoder, config.data.num_classes)

    # train setting
    criterion = get_loss(config)
    params = [
        {'params': model.base_params(), 'lr': config.optimizer.params.encoder_lr},
        {'params': model.fresh_params(), 'lr': config.optimizer.params.decoder_lr}
    ]
    optimizer = get_optimizer(params, config)
    scheduler = get_scheduler(optimizer, config)

    # model runner
    runner = SupervisedRunner(model=model)

    callbacks = [MultiClassAccuracyCallback(threshold=0.5), F1ScoreCallback()]

    if os.path.exists(config.work_dir + '/checkpoints/best.pth'):
        callbacks.append(CheckpointCallback(resume=config.work_dir + '/checkpoints/best_full.pth'))

    # model training
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        loaders=dataloaders,
        logdir=config.work_dir,
        num_epochs=config.train.num_epochs,
        callbacks=callbacks,
        verbose=True,
        fp16=True,
    )


def parse_args():
    parser = argparse.ArgumentParser(description='Severstal')
    parser.add_argument('--config', dest='config_file',
                        help='configuration filename',
                        default=None, type=str)
    return parser.parse_args()


def main(config_file):
    print('train Severstal Steel Defect Detection.')    
    if config_file is None:
        raise Exception('no configuration file')
    print('load config from {}'.format(config_file))
    run(config_file)

In [15]:
!ls config/cls

001_resnet50_BCE_5class_fold0.yml      003_seresnext50_cls_BCE_5class_fold2.yml
002_efnet_b3_cls_BCE_5class_fold1.yml


In [ ]:
# main('config/cls/001_resnet50_BCE_5class_fold0.yml')
# !cp -r 001_resnet50_BCE_5class_fold0 '/content/gdrive/My Drive/Steel Segmentation/'

In [ ]:
# main('config/cls/002_efnet_b3_cls_BCE_5class_fold1.yml')
# !cp -r 002_efnet_b3_cls_BCE_5class_fold1 '/content/gdrive/My Drive/Steel Segmentation/'

In [16]:
main('config/cls/003_seresnext50_cls_BCE_5class_fold2.yml')
!cp -r 003_seresnext50_cls_BCE_5class_fold2 '/content/gdrive/My Drive/Steel Segmentation/'

train Severstal Steel Defect Detection.
load config from config/cls/003_seresnext50_cls_BCE_5class_fold2.yml


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/checkpoints/se_resnext50_32x4d-a260b3a4.pth



1/40 * Epoch (train): 100% 1257/1257 [17:22<00:00,  1.21it/s, accuracy=0.967, f1_score=0.865, loss=0.061]
1/40 * Epoch (valid): 100% 315/315 [00:53<00:00,  5.87it/s, accuracy=1.000, f1_score=0.998, loss=8.200e-04]
[2020-07-14 08:25:19,527] 
1/40 * Epoch 1 (_base): lr=0.0001 | momentum=0.9000
1/40 * Epoch 1 (train): accuracy=0.9324 | f1_score=0.7609 | loss=0.1640
1/40 * Epoch 1 (valid): accuracy=0.9667 | f1_score=0.8841 | loss=0.0907
2/40 * Epoch (train): 100% 1257/1257 [17:23<00:00,  1.20it/s, accuracy=0.900, f1_score=0.691, loss=0.373]
2/40 * Epoch (valid): 100% 315/315 [00:53<00:00,  5.86it/s, accuracy=1.000, f1_score=0.999, loss=4.725e-04]
[2020-07-14 08:43:40,012] 
2/40 * Epoch 2 (_base): lr=0.0001 | momentum=0.9000
2/40 * Epoch 2 (train): accuracy=0.9639 | f1_score=0.8733 | loss=0.0932
2/40 * Epoch 2 (valid): accuracy=0.9682 | f1_score=0.8918 | loss=0.0850
3/40 * Epoch (train): 100% 1257/1257 [17:22<00:00,  1.21it/s, accuracy=1.000, f1_score=0.886, loss=0.053]
3/40 * Epoch (valid

In [ ]:
!ls